## Imports

In [1]:
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
import os
import warnings

warnings.simplefilter("ignore", category=UserWarning)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Environment

In [29]:
env = gym.make("Ant-v5", render_mode = "human")

In [30]:
env.reset()

(array([ 0.759666  ,  0.93612638,  0.05479718,  0.03089313, -0.07966557,
         0.0147533 ,  0.05673257, -0.05601841,  0.09381183, -0.07720832,
         0.09996778, -0.0654138 ,  0.08918836,  0.06748929, -0.11503934,
         0.0805477 ,  0.10240471, -0.01263313,  0.02198959,  0.10908869,
        -0.02427486,  0.21474775,  0.02506206, -0.15512165, -0.02010941,
         0.04755565,  0.13698456,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [31]:
env.action_space

Box(-1.0, 1.0, (8,), float32)

In [32]:
env.observation_space

Box(-inf, inf, (105,), float64)

In [33]:
env.render()

In [35]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset(seed=42)
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: -65.38229974698753
Episode: 2 Score: -2.051563134976405
Episode: 3 Score: -28.539221885457025
Episode: 4 Score: -42.50207586529442
Episode: 5 Score: -35.94617452445907


## Train Model

In [36]:
env = DummyVecEnv([lambda: env])

In [37]:
log_path = os.path.join("Training", "logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [38]:
model.learn(total_timesteps=4000000)

Logging to Training/logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 3570 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2939        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012003053 |
|    clip_fraction        | 0.127       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.3       |
|    explained_variance   | -0.0115     |
|    learning_rate        | 0.0003      |
|    loss                 | 36          |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0274     |
|    std                  | 0.993       |
|    value_loss           | 190         |
---

Use "tensorboard --logdir logs/" in the Training terminal to get to tensorboard

## Test Model

In [51]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(2642.8289833426943, 661.977017533995)

In [55]:
episodes = 5

for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: [2577.3853]
Episode: 2 Score: [2576.6895]
Episode: 3 Score: [2682.8376]
Episode: 4 Score: [2276.4602]
Episode: 5 Score: [2507.7488]
